In [1]:
import re
import time
import json
import uuid
import redis
import datetime
import pandas as pd

from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
best_seller_url = {
    'taobao':'https://s.taobao.com/search?initiative_id=tbindexz_20170306&ie=utf8&spm=a21bo.2017.201856-taobao-item.2&sourceId=tb.index&search_type=item&ssid=s5-e&commend=all&imgfile=&q=连衣裙&suggest=history_1&_input_charset=utf-8&wq=&suggest_query=&source=suggest&sort=sale-desc',
    'tmall':'https://list.tmall.com/search_product.htm?spm=a220m.1000858.1000724.4.62613ec2vteq0U&q=连衣裙&sort=d&style=g&from=.list.pc_1_searchbutton#J_Filter'
}

source_goods_url = {
    '17':'https://gz.17zwd.com/sks.htm?so=连衣裙&cateid=&display_name=&f_akmk=&color=&size=&mid=&sv=&fid=&range=&ord=&fs=&ml=&video=&image=&recomme=&pstart=&pend='
}

In [14]:
import configparser
cf = configparser.ConfigParser()
cf.read("config/crawl.cfg")

['config/crawl.cfg']

In [18]:
profile = webdriver.FirefoxProfile('/home/ml/.mozilla/firefox/0k5sh6ex.default')
driver = webdriver.Firefox(profile)

In [ ]:
key_words = '连衣裙'

final = pd.DataFrame()
for key,value in best_seller_url.items():
    url = value
    url = url.replace(re.findall('&q=(.*?)&',url)[0],key_words)
    print(url)
    platform_parse = key+'_parse_links'
    items_list = cf.options(platform_parse)
    items_list.pop(0)
    items_list.pop(0)
    
    driver.get(url)
    data = pd.DataFrame([],columns=items_list)
    # 等待信息装载
    for i in range(0, 40):
        driver.execute_script("document.documentElement.scrollTop="+str(i*100 + 100))
        time.sleep(0.1)
            
    # 提取信息
    while True:
        try:
            for item in items_list:
                if 'attribute' in item:
                    temp = cf.get(platform_parse, item).split('|')
                    a = driver.find_elements_by_xpath(temp[0])
                    data[item] = pd.Series([a[i].get_attribute(temp[1]) for i in range(0, len(a))])
                else:
                    a = driver.find_elements_by_xpath(cf.get(platform_parse, item))
                    data[item] = pd.Series([a[i].text for i in range(0, len(a))])
            data['id'] = pd.Series([str(uuid.uuid4()) for i in range(0, len(a))])
        except:
            continue
        break
        
    final = final.append(data).reset_index(drop=True)

final['key_words'] = key_words
final.to_csv('best_seller.csv', index=False)

In [19]:
key_words = '连衣裙'

time1 = time.time()
current_url = ''

platform_parse = '17_parse_links'
items_list = cf.options(platform_parse)
items_list.pop(0)
items_list.pop(0)

url = source_goods_url['17']
current_url = url
driver.get(url)
page_nums = int(driver.find_elements_by_xpath(cf.get(platform_parse,'page_num')[:-1])[0].text.split(cf.get(platform_parse,'page_num')[-1])[1])
page_nums = 20
final = pd.DataFrame([],columns=items_list)
data = pd.DataFrame([],columns=items_list)
for page_num in range(0,page_nums):
    # 等待信息装载
    while True:                   
        result = [len(driver.find_elements_by_xpath(cf.get(platform_parse, item))) for item in items_list]
        print(result)
        if (0 not in result)&(len(set(result)) == 1):
            for i in range(0, 50):
                driver.execute_script("document.documentElement.scrollTop="+str(i*200 + 200))
                time.sleep(0.05)
            break

    # 提取信息
    while True:
        try:
            for item in items_list:
                if 'attribute' in item:
                    temp = cf.get(platform_parse, item).split('|')
                    a = driver.find_elements_by_xpath(temp[0])
                    data[item] = pd.Series([a[i].get_attribute(temp[1]) for i in range(0, len(a))])
                else:
                    a = driver.find_elements_by_xpath(cf.get(platform_parse, item))
                    data[item] = pd.Series([a[i].text for i in range(0, len(a))])
            data['id'] = pd.Series([str(uuid.uuid4()) for i in range(0, len(a))])
            data['page_num'] = page_num
        except:
            continue
        break

    # 最后一页退出
    if page_num == (page_nums-1):
        break

    # 翻页,并等待页面刷新
    driver.find_elements_by_xpath(cf.get(platform_parse, 'click'))[0].click()
    while current_url == driver.current_url:
        pass

    # 超过一半重复退出
    current_num = len(final)
    final = final.append(data).drop_duplicates(items_list).reset_index(drop=True)
    if len(final) < current_num + int(len(data)/2):
        break
#     final.to_sql('links', con=engine,  if_exists='append', index=False)

print(time.time()-time1)
final.to_csv('source_goods.csv', index=False)

[103, 103, 103, 103, 103]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
[100, 100, 100, 100, 100]
148.61881470680237


In [6]:
import requests
from io import BytesIO
from PIL import Image

items = pd.read_csv('best_seller.csv')
for i in range(0,len(items)):
    try:
        if 'http' not in items['images_url_attribute'][i]:
            src = 'https:'+items['images_url_attribute'][i]
        else:
            src = items['images_url_attribute'][i]
        dir_path = 'data/best_seller/'+items['id'][i]+'.jpg'
        print(src)
        pic = Image.open(BytesIO(requests.get(src).content))
        pic = pic.convert('RGB')
        w, h = pic.size
        pic = pic.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
        pic.save(dir_path)
    except requests.exceptions.ConnectionError:
        print('Sorrry,image cannot downloaded, url is error{}.'.format(src))

https://g-search3.alicdn.com/img/bao/uploaded/i4/i2/2659008328/O1CN01cHwoMG2BOH3lyrIn4_!!0-item_pic.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i3/1746960915/O1CN01FWJ5Zb1Id71065AT5_!!1746960915-0-lubanu-s.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i4/2667435306/O1CN01zLrDiM1p4C77r3W0E_!!2667435306.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i1/2170746564/O1CN012dOgVi1yMMOHitUMw_!!2170746564.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i3/2264243541/O1CN01P2Tazr1c1p5E7F8Wu_!!0-item_pic.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i1/2003789956/O1CN01yxIMDM2NPtrLSX9ZL_!!0-item_pic.jpg
https://g-search1.alicdn.com/img/bao/uploaded/i4/i3/2024734791/O1CN01kDvnxI1lGKAZil85o_!!2024734791-0-lubanu-s.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i4/3818967491/O1CN010NjXTZ25CvTpmA3oA_!!0-item_pic.jpg
https://g-search2.alicdn.com/img/bao/uploaded/i4/i1/412288949/O1CN01vIhT862Fyh0iiH5wY_!!412288949-0-lubanu-s.jpg
https://g-search3.alicdn.com/img/

https://img.alicdn.com/bao/uploaded/i1/3307851244/O1CN019zjgk41L3nILAmMF6_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/3497134665/O1CN01PMg2kx1kKcGawLMxq_!!3497134665-0-lubanu-s.jpg
https://img.alicdn.com/bao/uploaded/i1/3258172357/O1CN017jqI2d1THY77kPAac_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i4/2206950006302/O1CN01ljRbnm1wQMaYhsF1K_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/2909151223/O1CN015Mzru11KuAzt8PEZR_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i2/745475881/O1CN01dP1CLo1tJXqnuFxbl_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/775029938/O1CN01DzPlqe2NHej0nY7Dm_!!775029938-0-lubanu-s.jpg
https://img.alicdn.com/bao/uploaded/i2/112394247/O1CN01VlNcKm1hFAiRAammw_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i1/2205433998/O1CN01LUeUjH1fP83l2MBuz_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i4/4215559603/O1CN01ssJvke2KoE2TQJSC1_!!0-item_pic.jpg
https://img.alicdn.com/bao/uploaded/i3/4130890307/O1CN0142wCtv1E8eCzG

In [20]:
items = pd.read_csv('source_goods.csv')
for i in range(0,len(items)):
    try:
        if 'http' not in items['images_url_attribute'][i]:
            src = 'https:'+items['images_url_attribute'][i]
        else:
            src = items['images_url_attribute'][i]
        dir_path = 'data/source_goods/'+items['id'][i]+'.jpg'
        print(src)
        pic = Image.open(BytesIO(requests.get(src).content))
        pic = pic.convert('RGB')
        w, h = pic.size
        pic = pic.resize((int(w / 2), int(h / 2)), Image.ANTIALIAS)
        pic.save(dir_path)
    except requests.exceptions.ConnectionError:
        print('Sorrry,image cannot downloaded, url is error{}.'.format(src))

https://img.alicdn.com/bao/uploaded/i1/747504891/O1CN01Bnaa5c1m07lHIPEBr_!!747504891.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61857039/i1/684347072777814016.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1608188013/O1CN01FN2cAD2940HPJzWmT_!!1608188013.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/222189/i1/706665034679320576.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/224416/i1/643123564995674112.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/219270/i1/626810470216499200.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/439785740/O1CN011sGxs9njqsF2jpi_!!439785740.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2446874276/O1CN01eMWEv21hSSCgXj2B1_!!2446874276.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/216030/i1/cb21b71c-f222-4f06-a7db-32510bcc9949.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/23327/i1/8b063cd7-4e0c-41e2-a74a-127a93e7a1e0.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2776276664/O1CN01OCwxDj1z6

https://img.alicdn.com/bao/uploaded/i4/1605892507/O1CN01gqNm611UOFXAN37P0_!!1605892507.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2256175393/O1CN01RxtXKv1pi2bi4KSEy_!!2256175393.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1052024726/O1CN016kecEK1kmYPJvHe9T_!!1052024726.png_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61414889/i1/704118313277456384.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/204840245/O1CN01SxKhw71DgFcEBIeoR_!!204840245.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2256175393/O1CN01ZznhMs1pi2bBXU28Y_!!2256175393.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/240141/i1/e44f7fab-bdfb-4ffc-bb06-881c113f9097.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/240141/i1/3b3d4c95-991f-4fb7-9f39-2774774cb455.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/240141/i1/aebca6bc-a6f2-4926-9d32-8b513720145d.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1605892507/O1CN01SbtQVJ1UOFXAVPOtt_!!1605892507.jpg_600x600.jpg


https://img.alicdn.com/bao/uploaded/i1/204840245/O1CN01rgqxbG1DgFeOdiVZx_!!204840245.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1052024726/O1CN01HR0HGe1kmYQ6kOtEC_!!1052024726.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1052024726/O1CN01n0Bj7S1kmYQDuDm8q_!!1052024726.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2965372543/O1CN01osYmGr1UejnpsR82j_!!2965372543.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61234312/i1/692522067609780224.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/836998723/O1CN01j81nVw2EJBTnySudL_!!836998723.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/889412042/TB2fnjSGntYBeNjy1XdXXXXyVXa_!!889412042.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/240141/i1/67f009b4-ce64-484e-9905-1f08dd677b03.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1052024726/O1CN01X2GBtM1kmYPafADfb_!!1052024726.png_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2771435482/O1CN01csdqlq1qMnqqNNM5y_!!2771435482.jpg_600x60

https://img01-gms.17zwd.com/imgextra/293624/i1/72bffb04-7865-41d5-be75-4e4e760d422d.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1605892507/O1CN01dgBWMw1UOFYZDdjgP_!!1605892507.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2378061686/O1CN01RJNJa01OKENqfJM9k_!!0-item_pic.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/925307001/O1CN01FhEQ0U21aVTRlepqW_!!925307001.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2256175393/O1CN01Swenc71pi2bbexO3x_!!2256175393.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/204840245/O1CN01r69Nmk1DgFenVE4an_!!204840245.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2256175393/O1CN01D4jNaO1pi2bm4cxsN_!!2256175393.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/204840245/O1CN01SgefsR1DgFeifCfkp_!!204840245.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1848064547/O1CN01KgBTRQ1jSZW0hGt51_!!0-item_pic.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1036218965/O1CN01NR7JwJ2G61JtYvaNL_!!103621

https://img.alicdn.com/bao/uploaded/i1/2256175393/O1CN01Y7YqSn1pi2bopU5rD_!!2256175393.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2122567480/O1CN018skVYj257tCRCTQs2_!!2122567480.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/439785740/O1CN01jBMlhC1sGy2ogI7PJ_!!439785740.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/439785740/O1CN01QeKzYg1sGy2vhi6Ba_!!439785740.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/715299391/O1CN01MRHkrl2JF84vzCNx3_!!715299391.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1680073052/O1CN0179wT4q1YPrQ99dKH1_!!1680073052.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1829510523/O1CN01Wjc7d31FjZli1U0bO_!!1829510523.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/1612402479/O1CN01ZbGhEy1UBQTV3ITtS_!!1612402479.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1772126189/O1CN01oxHPal1vabsayrjWC_!!1772126189.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2855439657/O1CN01hjEpXD2LCxR2gqlKX_!!285

...

https://img01-gms.17zwd.com/imgextra/61310010/i1/1adcea49-6576-4333-a341-cd022f9d6b74.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1645543336/O1CN01tilUs21aVvu2GgNJZ_!!1645543336.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/514012/i1/7546c052-ab42-4bd2-8a8b-6b5104020df8.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1066665754/O1CN01EWkFwE1sNNZg5SaQR_!!1066665754.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2059753986/O1CN01VRegSK1fJdLUDdwFg_!!2059753986.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/794473396/O1CN018xU6Kw1axPfGeK08O_!!794473396.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/3066370146/O1CN01rNv1AV1CwuS0plc9Q_!!3066370146.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2492839665/O1CN01C2Xluw2LGcbPhbgQ9_!!2492839665.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/62133418/i1/24ab21bd-e2f0-41ad-8ed0-2742e814394f.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61414015/i1/fa7a22b9-4956-4281-8585-ccb8b2e

https://img01-gms.17zwd.com/imgextra/61218584/i1/c48da659-9478-4854-8c8c-b1dab861b0e7.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/2486307384/O1CN01HzF3Q524PvAtYxDLz_!!2486307384.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/850312650/O1CN013mEYhy1VRkCVbHpdV_!!850312650.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/275154400/O1CN01ngjjUF1iNFJg0LdUy_!!275154400.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/109958571/O1CN01T8Cw5P2DBZJ6UhqB3_!!109958571.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/3903469610/O1CN016cNVqT2KrQobH4DYc_!!3903469610.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61243737/i1/8e82323e-e04c-49e6-a86a-3a355deffcbb.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61925318/i1/02214c7e-17c8-40f6-ab74-8bff8362c2f3.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/420714914/O1CN01wufX881mAejrsOQQl_!!420714914.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/888944061/O1CN01ZNVQ491frz39s9IA3_!!888944061

https://img.alicdn.com/bao/uploaded/i2/836998723/O1CN01gBSX3d2EJBUpDi9xc_!!836998723.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61409383/i1/2a1c401a-d805-4949-a8be-115dacaa797b.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/3914358423/O1CN012HzKUJ2C5mXBXVast_!!3914358423.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/2087027860/O1CN01Unsmru27vvhhsdVRX_!!2087027860.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/925307001/O1CN01ev4VIq21aVT7hRTH5_!!925307001.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/789162338/O1CN01FuvJat1T8qd0c3f3v-789162338.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2561480715/O1CN01GJUgML1H9VoVAAYlr_!!2561480715.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61911509/i1/a08f0b63-4ecb-45a6-95ab-72445302ee03.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/352023223/O1CN014erlmf1ZgB5MNUFUZ_!!352023223.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/11931308/O1CN01vp3BeR1LX6gIL9kxU_!!11931308.jp

https://img.alicdn.com/bao/uploaded/i3/2707857554/O1CN01HZJFu525fmVZ1szQZ_!!2707857554.png_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/893025793/O1CN01KYmMD81sfF2MzCG42_!!893025793.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/68637641/O1CN01wGib1i26Jcxvmz3Jk_!!68637641.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/3030152296/O1CN01qRFq1X1SpbzhL25uJ_!!3030152296.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61298059/i1/c18f5dbc-b426-4097-8908-343935e07e88.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/1772126189/O1CN01QkJJdh1vabsS0J26c_!!1772126189.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1820989369/O1CN01frTqNr2J53Mv7uIvd_!!1820989369.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1820989369/O1CN01Fec3pB2J53MmLNiaH_!!1820989369.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/699675445/O1CN01OjOLYs1q5rCqTIsDM_!!699675445.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1755302331/O1CN01CXUfdr1T5drBdwekV_!!175530

https://img01-gms.17zwd.com/imgextra/498768/i1/ae726210-3256-4543-96e7-ce8694cf7851.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61298059/i1/3f2f4175-5f10-465d-b7a9-da4bb2f3d3ea.jpg_600x600.jpg
https://img01-gms.17zwd.com/imgextra/61298059/i1/8e8abde3-1777-44e0-9767-a357ea95267a.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/2341542092/O1CN01PuzCip1RKBCFNaw7j_!!2341542092.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i3/2231109733/O1CN01iUp1pj2LllXKWHMEe_!!2231109733.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1683899021/O1CN01M1RuVY2GVfX8ZIjYH_!!1683899021.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i4/894180229/O1CN01pAxPfW1DYvKMWUxlj_!!894180229.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/1645543336/O1CN012CTl7v1aVvu21xI8F_!!1645543336.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i1/354468624/O1CN01e0Af9b2DZqIYwKgPY_!!354468624.jpg_600x600.jpg
https://img.alicdn.com/bao/uploaded/i2/1597185844/O1CN01uOAziP1t2bEfHrlFH_!!159718